<a href="https://colab.research.google.com/github/aureliosaraiva/UniversidadEuropeaTFM/blob/main/tfm_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets sentence-transformers faiss-cpu bitsandbytes -q
!pip install --upgrade datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import torch
from google.colab import files
from datasets import load_dataset
from transformers import BloomTokenizerFast, BloomForCausalLM, Trainer, TrainingArguments
from datasets import Dataset

os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
from google.colab import files
import os

uploaded = files.upload()

for nome_original in uploaded.keys():
    novo_nome = 'dataset_raw.ndjson'
    os.rename(nome_original, novo_nome)

Saving dataset.ndjson to dataset.ndjson


In [5]:
from google.colab import files
import json
import os

# Função para extrair os campos importantes de cada objeto JSON
def extract_fields(service_obj):
    """
    Extrai os campos:
      - name: metadata.name
      - description: metadata.description
      - dependsOn: se existir o campo 'dependsOn'; caso contrário, utiliza spec.consumesApis
      - labels: metadata.labels
    """
    metadata = service_obj.get("metadata", {})
    name = metadata.get("name", "")
    description = metadata.get("description", "")
    labels = metadata.get("labels", {})

    # Se não houver 'dependsOn', tenta usar spec.consumesApis
    spec = service_obj.get("spec", {})
    dependsOn = service_obj.get("dependsOn", spec.get("consumesApis", []))
    # Garante que dependsOn seja uma lista
    if not isinstance(dependsOn, list):
        dependsOn = [dependsOn]

    return {
        "name": name,
        "description": description,
        "dependsOn": dependsOn,
        "labels": labels
    }


def extract_fields_to_text(service_obj):
    """
    Extrai os campos:
      - name: metadata.name
      - description: metadata.description
      - dependsOn: se existir o campo 'dependsOn'; caso contrário, utiliza spec.consumesApis
      - labels: metadata.labels
    """
    metadata = service_obj.get("metadata", {})
    name = metadata.get("name", "")
    description = metadata.get("description", "")
    labels = metadata.get("labels", {})

    # Se não houver 'dependsOn', tenta usar spec.consumesApis
    spec = service_obj.get("spec", {})
    dependsOn = service_obj.get("dependsOn", spec.get("consumesApis", []))
    # Garante que dependsOn seja uma lista
    if not isinstance(dependsOn, list):
        dependsOn = [dependsOn]
    return f"""
Microservice Info:
Name: {name}
Description: {description}
Dependencies: {", ".join(dependsOn)}
Labels: {", ".join(labels)}
    """

# Lê o arquivo NDJSON linha a linha e converte cada linha para um objeto JSON
data = []
with open("dataset_raw.ndjson", "r") as f:
    for line in f:
        line = line.strip()
        if line:  # ignora linhas vazias
            data.append(json.loads(line))

def save_to_json():
    dataset = [extract_fields(service) for service in data]
    output_filename = "dataset_output.ndjson"
    if os.path.exists(output_filename):
        os.remove(output_filename)
    with open(output_filename, "w") as f:
        for record in dataset:
            f.write(json.dumps(record) + "\n")

    print(f"Arquivo JSON '{output_filename}' criado com sucesso!")

def save_to_text():
    dataset = [extract_fields_to_text(service) for service in data]
    output_filename = "dataset_output.text"
    if os.path.exists(output_filename):
        os.remove(output_filename)
    with open(output_filename, "w") as f:
        for record in dataset:
            f.write(json.dumps(record) + "\n")

    print(f"Arquivo TEXT '{output_filename}' criado com sucesso!")

save_to_json()
save_to_text()

Arquivo JSON 'dataset_output.ndjson' criado com sucesso!
Arquivo TEXT 'dataset_output.text' criado com sucesso!


In [ ]:
def extract_info(entry):
    return {
        'name': entry['metadata']['name'],
        'description': entry['metadata'].get('description', ''),
        'owner': entry['spec']['owner'],
        'tribe': entry['metadata']['annotations'].get('creditas.io/tribe', ''),
        'language': entry['metadata']['annotations'].get('creditas.io/language', ''),
        'dependencies': entry['spec'].get('dependsOn', []),
        'tags': entry['metadata'].get('tags', []),
        'full_context': str(entry)  # Contexto completo para o QA
    }

df = pd.DataFrame([extract_info(c) for c in data])

# Salvar contexto completo para retrieval
df['full_context'].to_csv('contextos_microservices.csv', index=False)


In [ ]:
df

,name,description,owner,tribe,language,dependencies,tags,full_context
0,aang,Responsible for installments conciliation with...,tribe-benefits,tribe-benefits,Kotlin,"[component:authentication, component:lego, com...","[benefits, kotlin, auto-commit, service]",{'metadata': {'annotations': {'backstage.io/ma...
1,account-recovery,The service responsible for recovering custome...,tribe-communication,tribe-communication,kotlin,"[component:alexandria, component:authentication]",[account-recovery],{'metadata': {'annotations': {'backstage.io/ma...
2,acquisition,"Formerly known as mediator, acquisition is the...",tribe-acquisition,tribe-acquisition,Kotlin,[],[service],{'metadata': {'annotations': {'backstage.io/ma...
3,acquisition-form-builder,Application responsible to ...,tribe-acquisition,tribe-acquisition,typescript,[],[microfrontend],{'metadata': {'annotations': {'backstage.io/ma...
4,acquisition-simulator-bff,A BFF responsible for centralize all services ...,conversion-devs,,,[],[service],{'metadata': {'annotations': {'backstage.io/ma...
...,...,...,...,...,...,...,...,...
745,website-assets,"Repository for static assets (css, fonts) of E...",tribe-acquisition,tribe-acquisition,HTML,[],[],{'metadata': {'annotations': {'backstage.io/ma...
746,whatsapp-border,Microservice designed to handle with whatsapp ...,tribe-communication,tribe-communication,Kotlin,"[component:individual-api, component:conversat...",[iris-platform],{'metadata': {'annotations': {'backstage.io/ma...
747,whatsapp-business-atwork,Conector que le as mensagem de whatsapp e enca...,tribe-benefits,tribe-benefits,TypeScript,"[component:benefits-leads-salesforce, componen...","[typescript, auto-commit]",{'metadata': {'annotations': {'backstage.io/ma...
748,wiremock,Wiremock server configured with several HTTP r...,tribe-engineering,tribe-engineering,HCL,[],[],{'metadata': {'annotations': {'backstage.io/ma...


In [ ]:
qa_pairs = []

for _, row in df.iterrows():
    context = str(row['full_context'])

    # Gerar perguntas automáticas baseadas na estrutura
    qa_pairs.append({
        'question': f"Qual é o dono do serviço {row['name']}?",
        'answer': row['owner'],
        'context': context
    })

    qa_pairs.append({
        'question': f"Que linguagem é usada no {row['name']}?",
        'answer': row['language'],
        'context': context
    })

    qa_pairs.append({
        'question': f"Lista as dependências do serviço {row['name']}",
        'answer': ', '.join(row['dependencies']),
        'context': context
    })

    qa_pairs.append({
        'question': f"Quais tags estão associadas ao {row['name']}?",
        'answer': ', '.join(row['tags']),
        'context': context
    })

# Converter para DataFrame
qa_df = pd.DataFrame(qa_pairs)

In [6]:
with open("dataset_raw.ndjson", "r") as file:
    dados = [json.loads(line) for line in file]

dataset_raw = Dataset.from_dict({"text": dados})

In [29]:
dataset_raw

Dataset({
    features: ['text'],
    num_rows: 750
})

In [7]:
# =============================================================================
# 3. Formatação dos Exemplos para Treinamento
# =============================================================================
# Para cada registro, criamos um prompt que reúne os campos extraídos.
def format_prompt(example):
    component = example['text']['metadata']

    name = component.get("name", "N/A")
    description = component.get("description", "N/A")
    dependsOn = component.get("dependsOn", [])
    if isinstance(dependsOn, list):
        dependsOn = ", ".join(dependsOn)
    labels = component.get("labels", {})
    # Utiliza json.dumps com default=str para converter objetos não serializáveis (por exemplo, datetime)
    prompt = (
        f"Microservice Info:\n"
        f"Name: {name}\n"
        f"Description: {description}\n"
        f"Dependencies: {dependsOn}\n"
        f"Labels: {json.dumps(labels, default=str)}\n\n"
        f"Treinamento:"
    )
    return {"text": prompt}

# Aplica a função de formatação ao dataset.
dataset = dataset_raw.map(format_prompt)

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [50]:
dataset.data

InMemoryTable
text: string
----
text: [["Microservice Info:
Name: aang
Description: Responsible for installments conciliation with the B2B partners
Dependencies: 
Labels: {"creditas.io/cliditas-api-version": "v2alpha1", "creditas.io/cliditas-version": "v0.9.35", "creditas.io/database-version": null, "creditas.io/disaster-recovery": null, "creditas.io/docker-image": "openjdk-13-slim-buster", "creditas.io/eventlib": "2.16.0-rc.1", "creditas.io/gradle-version": "7.6.3", "creditas.io/impact": null, "creditas.io/java-version": "1.8", "creditas.io/kotlin-jvm-version": "1.4.30", "creditas.io/language": "Kotlin", "creditas.io/last-updated": "2025-01-09", "creditas.io/microservice-toolkit": null, "creditas.io/pcn": null, "creditas.io/sentry": null, "creditas.io/severity-critital": "0", "creditas.io/severity-high": "0", "creditas.io/severity-low": "0", "creditas.io/severity-moderate": "0", "creditas.io/spring-auth": null, "creditas.io/spring-boot-version": "2.2.1.RELEASE", "creditas.io/tribe": n

In [8]:
# =============================================================================
# 4. Tokenização dos Exemplos
# =============================================================================
# Carrega o tokenizer do modelo BLOOM (usando a variante menor para economizar espaço)
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-560m")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [9]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True, max_length=512, padding="max_length")
    # Define os labels como uma cópia dos input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=False)
print("Exemplo tokenizado:", tokenized_dataset[0])

split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Exemplo tokenizado: {'text': 'Microservice Info:\nName: aang\nDescription: Responsible for installments conciliation with the B2B partners\nDependencies: \nLabels: {"creditas.io/cliditas-api-version": "v2alpha1", "creditas.io/cliditas-version": "v0.9.35", "creditas.io/database-version": null, "creditas.io/disaster-recovery": null, "creditas.io/docker-image": "openjdk-13-slim-buster", "creditas.io/eventlib": "2.16.0-rc.1", "creditas.io/gradle-version": "7.6.3", "creditas.io/impact": null, "creditas.io/java-version": "1.8", "creditas.io/kotlin-jvm-version": "1.4.30", "creditas.io/language": "Kotlin", "creditas.io/last-updated": "2025-01-09", "creditas.io/microservice-toolkit": null, "creditas.io/pcn": null, "creditas.io/sentry": null, "creditas.io/severity-critital": "0", "creditas.io/severity-high": "0", "creditas.io/severity-low": "0", "creditas.io/severity-moderate": "0", "creditas.io/spring-auth": null, "creditas.io/spring-boot-version": "2.2.1.RELEASE", "creditas.io/tribe": null}\n\

In [10]:
# =============================================================================
# 5. Carregar o Modelo (Variante Menor com Quantização 8-bit se possível)
# =============================================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    # Utiliza quantização 8-bit para reduzir o uso de memória/disco
    model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")
else:
    model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")
model.to(device)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [11]:
# =============================================================================
# 6. Configuração do Treinamento
# =============================================================================


from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss
10,2.098500
20,1.082400
30,0.796700
40,0.507300
50,0.839900
60,0.719600
70,0.555600
80,0.424300
90,0.429300
100,0.924100


TrainOutput(global_step=1125, training_loss=0.3644759618971083, metrics={'train_runtime': 783.0168, 'train_samples_per_second': 2.874, 'train_steps_per_second': 1.437, 'total_flos': 2089590718464000.0, 'train_loss': 0.3644759618971083, 'epoch': 3.0})